In [17]:
import pickle
import numpy as np

In order to slowly move towards a modular file structure, we define constants, the model parameters, and ohters in separate notebooks. This notebook uses the ipynb library (pip install ipynb) to import constants, model parameters, and funtions that have been moved to other notebooks. The goal of this restructuring is to improve readability, make dependences clear and ease settig up testing tear.

In [18]:
# Execute entire file and make all variables/functions/classes
# available for further use
from ipynb.fs.full.model_spec import num_periods, num_choices, educ_max, educ_min

# Import specified definitions only from given notebook
import ipynb.fs
from .defs.shared_constants import MISSING_INT

In [15]:
num_periods

10

In [4]:
# Auxiliary calculation of the education dimension
educ_range = educ_max - educ_min + 1

Next, we create arrays which we want to populate with the state space variables.
The components of our state space are:
- the time variable: period
- the initial conditions: education
- the history of choices: choice_lagged
- the accumulated experience in part-time employment: exp_p
- the accumulated experience in full-time employment: exp_f

In [6]:
# Array for mapping the state space points to indices
shape = (num_periods, educ_range, num_choices, num_periods, num_periods)
mapping_state_index = np.tile(MISSING_INT, shape)

# Maximum number of state space points per period. There can be no more states in a period than this number. 
num_states_period_upper_bound = np.prod(mapping_state_index.shape)
print(mapping_state_index.shape, num_states_period_upper_bound)

(10, 5, 3, 10, 10) 15000


Let us briefly discuss the dimension of the mapping_state_index array. Each dimension component corresponds to the state space component we would like to keep record of.
- dimension 1 num_periods corresponds to the period number
- educ_range to the years of education
- num_choices to the number of choices available to the agents each period
- dimension 4 num_periods to years of experience in part-time
- dimension 5 num_periods to years of experience in full-time

In [7]:
# Array to collect all state space points (states) that can be reached each period
states_all = np.tile(MISSING_INT, (num_periods, num_states_period_upper_bound, 4))

Let us briefly discuss the dimention of the states_all array:
- the 1st dimension is determined by the number of periods in our model
- the 2nd dimension is related to the maximum number of state space points ever feasible / possibly reachable in one of the periods. *Question: Is this true? Is 100 000 simply some arbitrary large number that is for sure larger than the highest possible number of state space points ever reachable in a period? Can one replace this number by some educated guess of the maximum number of state point combinantions given no restrictions?*
- the 3rd dimension is equal to the number of remaining state space components (except period number) that we want to record: educ_years + educ_min, choice_lagged, exp_p, exp_f

In [8]:
# Array for the maximum number state space points / states per period
states_number_period = np.tile(MISSING_INT, num_periods)

In a final step, we loop through all admissible state space points and fill up the constructed arrays with necessary information.

Introducing education as initial condition:

- If we want to record only admissible state space points, we have to account for the fact that individuals in the model start making labor supply choices only after they have completed education.

Let us look into latter in greater detail. As a reminder, we note that we model individuals from age 16 (legally binding mininum level of education) until age 60 (typical retirement entry age), i.e., for 45 periods, where the 1st period corresponds to age 16, the second period to age 17, etc. The current specification of starting values is equivalent to the observation / simulated reality that individuals in the sample have completed something between 10 and 14 years of edcuation. In our loop we want to take into account the fact that, in the first period at age 16, only individuals who have completed 10 years of education (assuming education for everyone starts at the age of 6) will be making a labor market choice between full-time (F), part-time (P), and non-employment (N). The remaining individuals are still in education, such that a state space point where years of education equal e.g. 11 and a labor market choice of e.g. part-time is observed is not admissible and should therefore not be recorded. This is ensured by the if clause "educ_years > period".

- When starting the model in the way described, we need to account for an additional state space point which corresponds to individuals who have just entered the model.

Let us consider the very first period, period 0 in the loop, as an example. In this period, only individuals with 10 years of education enter the model and make their first labor supply choice. In order to later be able to compute the instataneous utility for this individuals, we need to record their state space components. As there is no lagged choice available to record, we force set the lagged choice component to zero. In particular, for states for which the condition (period == educ_years) is true, i.e., for individuals who have just entered the model, we record a state: (educ_years + educ_min, 0, 0, 0).

In [9]:
# Loop over all periods / all ages
for period in range(num_periods):
    
    # Start count for admissible state space points
    k = 0
        
    # Loop over all possible initial conditions for education
    for educ_years in range(educ_range):
            
        # Check if individual has already completed education
        # and will make a labor supply choice in the period
        if educ_years > period:
            continue
        
        # Loop over all admissible years of experience accumulated in part-time
        for exp_f in range(num_periods):
            
            # Loop over all admaissible years of experience accumulated in full-time
            for exp_p in range(num_periods):
                
                # The accumulation of experience cannot exceed time lapsed
                # since individual entered the model
                if (exp_f + exp_p > period - educ_years):
                    continue
                
                # Add an additional entry state [educ_years + educ_min, 0, 0, 0]
                # for individuals who have just completed education
                # and still have no experience in any occupation.
                if (period == educ_years):

                    # Assign an additional the integer count k
                    # for entry state
                    mapping_state_index[
                        period,
                        educ_years,
                        0,
                        0,
                        0,
                    ] = k

                    # Record the values of the state space components
                    # for the currentry reached entry state
                    states_all[period, k, :] = [
                        educ_years + educ_min,
                        0,
                        0,
                        0,
                    ]

                    # Update count once more
                    k += 1

                else:
                
                    # Loop over the three labor market choices, N, P, F
                    for choice_lagged in [1,2,3]:

                        # If individual has only worked full-time in the past,
                        # she can only have full-time (3) as lagged choice
                        if (choice_lagged != 3) and (exp_f == period - educ_years):
                            continue

                        # If individual has only worked part-time in the past,
                        # she can only have part-time (2) as lagged choice
                        if (choice_lagged != 2) and (exp_p == period - educ_years):
                            continue

                        # If an individual has never worked full-time,
                        # she cannot have that lagged activity
                        if (choice_lagged == 3) and (exp_f == 0):
                            continue

                        # If an individual has never worked part-time,
                        # she cannot have that lagged activity
                        if (choice_lagged == 2) and (exp_p == 0):
                            continue

                        # Check for duplicate states
                        if (
                            mapping_state_index[
                                period,
                                educ_years,
                                choice_lagged - 1,
                                exp_p,
                                exp_f,
                            ]
                            != MISSING_INT
                        ):
                            continue

                        # Assign the integer count k as an indicator for the
                        # currently reached admissible state space point
                        mapping_state_index[
                            period,
                            educ_years,
                            choice_lagged - 1,
                            exp_p,
                            exp_f,
                        ] = k

                        # Record the values of the state space components
                        # for the currently reached admissible state space point
                        states_all[period, k, :] = [
                            educ_years + educ_min,
                            choice_lagged -1,
                            exp_p,
                            exp_f,
                        ]

                        # Update count
                        k += 1
    
    # Record number of admissible state space points for the period currently reached in the loop 
    states_number_period[period] = k

We briefly repeat what has been recorded in the states_all array:
 - educ_years + educ_min: in this example, values from 10 to 14
 - choice_lagged -1: 0, 1, 2 corresponding to N, P, and F
 - exp_p: part-time experience that can range from 0 to 9
 - exp_f: full-time experience that can range from 0 to 9
 
 *Note: There is a difference to respy here. In respy, the loop in experience is one iteration longer, goes to num_periods + 1 instead of to num_periods.*

In [10]:
# Auxiliary objects
max_states_period = max(states_number_period)

In [11]:
# Collect arguments
args = (states_all, states_number_period, mapping_state_index, max_states_period)

In [12]:
states_number_period

array([  1,   4,  14,  34,  67, 115, 180, 260, 355, 465])

Ideas checks and tests:
- check if loop visits the whole range of values by checking min and max of array entries for the differnt state space components.

In [13]:
print(np.amax(states_all[:,:,0]), np.amax(states_all[:,:,1]), np.amax(states_all[:,:,2]), np.amax(states_all[:,:,3]))

14 2 9 9


Export final output:

In [14]:
# Choose a file name
file_name = "args_file.pkl"

# Open the file for writing
with open(file_name,'wb') as my_file_obj:
    pickle.dump(args, my_file_obj)   